## WeatherPy: Visualizing weather based on Latitude  
March 2, 2019  
Scott McEachern

In [1]:
#-- Import Libraries
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import requests

# OpenWeatherMap API Key located in secrets.py; .gitignore prevents secrets.py from push to GitHub
from secrets import api_key

# citipy library to lookup city name based on lat/long; 
# require installation of dependencies with: pip install citipy
from citipy import citipy


In [2]:
#-- Generate List of Cities

# Number of random locations to calculate
numRandomLocations = 80

# Total number of cities to get weather information from API
numCities = 50


#- Create Random Coordinates
lats = np.random.uniform(low=-90.000, high=90.000, size=numRandomLocations)
lngs = np.random.uniform(low=-180.000, high=180.000, size=numRandomLocations)


#- Merge into Zip
lat_lngs = zip(lats, lngs)


#- Find City Name
cityNames = []

for lat_lng in lat_lngs:
    
    # Get nearest city
    cityName = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    
    # Determine new city name
    if (cityName not in cityNames):
        cityNames.append(cityName)

        # Check if reached number of cities used with analysis
        if (len(cityNames) == numCities):
            break
               

In [3]:
def getWeatherForCity(cityName):
    ''' Queries the OpenWeatherData API to get the current weather conditions for the city name provided.
    Exceptions are captured and when exception, metadata printed to console.
    
    Accepts : cityName (str) Name of the city to search to get current weather information for
    
    Returns : List - contains metadata from the API; returns None when exception or not 200 response
                0 - (str) City Name
                1 - (str) Country Code
                2 - (num) Latitude
                3 - (num) Longitude
                4 - (num) Max Temperature - F
                5 - (num) Windspeed - MPH
                6 - (num) Humidity - %
                7 - (num) Cloudiness - %
    '''
    
    try:
        
        #- Prepare URL
        baseWeatherUrl = "http://api.openweathermap.org/data/2.5/weather"
        unitFormat = "imperial"
        
        queryUrl = f"{baseWeatherUrl}?q={cityName}&units={unitFormat}&appid={api_key}"
        
        
        #- Get Weather from API
        response = requests.get(queryUrl)
        
        
        #- Check Response
        cityWeather = []
        
        if (response.status_code == requests.codes.ok):
            
            # Get Json from Response
            responseJson = response.json()
            
            
            # Get Weather Information
            cityWeather.append(responseJson['name'])
            cityWeather.append(responseJson['sys']['country'])
            
            cityWeather.append(responseJson['coord']['lat'])
            cityWeather.append(responseJson['coord']['lon'])
            
            cityWeather.append(responseJson['main']['temp_max'])
            cityWeather.append(responseJson['wind']['speed'])
            
            cityWeather.append(responseJson['main']['humidity'])
            cityWeather.append(responseJson['clouds']['all'])
        
        else:
            #- Response not valid; return None
            return
        
        
        return cityWeather
        
    except Exception as e:
        print(f"   Exception getting weather from API. City: {cityName} Error Type: {type(e)} Info: {e}")

In [4]:
##- Get Weather for City

#- Prepare Dictionary to store results
weatherData = {
    'City': [],
    'Country': [],
    'Latitude' : [],
    'Longitude' : [],
    'Temperature' : [],
    'WindSpeed' : [],
    'Humidity' : [],
    'Cloudiness': [],
    }


#- Display Message
print("> Start data retrieval")


#- Get Weather Information
errorCounter = 0
counter = 0

for cityName in cityNames:
    
    # Progress Message
    counter += 1
    
    print(f"Processing: {counter} of {len(cityNames)} - {cityName.title()}")
    
    
    # Get Weather for City
    cityInfo = getWeatherForCity(cityName)
    
    
    # Check Results from API
    if (cityInfo != None):
        weatherData['City'].append(cityInfo[0])
        weatherData['Country'].append(cityInfo[1])
        weatherData['Latitude'].append(cityInfo[2])
        weatherData['Longitude'].append(cityInfo[3])
        weatherData['Temperature'].append(cityInfo[4])
        weatherData['WindSpeed'].append(cityInfo[5])
        weatherData['Humidity'].append(cityInfo[6])
        weatherData['Cloudiness'].append(cityInfo[7])
        
    else:
        print(f"   Unable to get weather information for city: {cityName.title()}")
        errorCounter +=1
    
    
#- Display Message
print(" ")
print(f"Completed getting data. Total Cities: {len(weatherData['City'])}  Error Cities: {errorCounter}")
  

> Start data retrieval
Processing: 1 of 50 - Sarangani
Processing: 2 of 50 - Santa Cruz
Processing: 3 of 50 - Arraial Do Cabo
Processing: 4 of 50 - Matara
Processing: 5 of 50 - Praia Da Vitoria
Processing: 6 of 50 - Ibra
Processing: 7 of 50 - Bethel
Processing: 8 of 50 - Ushuaia
Processing: 9 of 50 - Lebu
Processing: 10 of 50 - Cape Town
Processing: 11 of 50 - Narsaq
Processing: 12 of 50 - Nome
Processing: 13 of 50 - Avarua
Processing: 14 of 50 - Ruatoria
   Unable to get weather information for city: Ruatoria
Processing: 15 of 50 - Berlevag
Processing: 16 of 50 - Talesh
   Unable to get weather information for city: Talesh
Processing: 17 of 50 - Atuona
Processing: 18 of 50 - Saint-Philippe
Processing: 19 of 50 - Mataura
Processing: 20 of 50 - Mochalishche
Processing: 21 of 50 - Bluff
Processing: 22 of 50 - Kazalinsk
   Unable to get weather information for city: Kazalinsk
Processing: 23 of 50 - Hede
Processing: 24 of 50 - Salinas
Processing: 25 of 50 - Dwarka
Processing: 26 of 50 - Hi

In [5]:
#-- Prepare Data For Charting

#- Create Data Frame
weather_df = pd.DataFrame(weatherData)

weather_df.head()

,City,Country,Latitude,Longitude,Temperature,WindSpeed,Humidity,Cloudiness
0,Sarangani,PH,5.40,125.46,79.23,9.42,100,8
1,Santa Cruz,US,36.97,-122.03,59.00,6.93,87,90
2,Arraial do Cabo,BR,-22.97,-42.02,91.40,8.05,59,40
3,Matara,PE,-13.74,-72.90,37.40,9.17,328,75
4,Praia da Vitoria,PT,38.73,-27.07,60.80,15.35,59,75
